In [1]:
import json

import polars as pl
from polars import col

In [2]:
# Initial data cleaning (with steam_games.parquet and steam_games_extra_features.parquet)
# steam_games_agg = steam_games.join(steam_game_features, on="appid", how="left")
# steam_games_agg = steam_games_agg.with_columns(
#     pl.col("review_score").fill_null(0),
#     pl.col("review_score_desc").fill_null("No reviews"),
#     pl.col("total_positive_reviews").fill_null(0),
#     pl.col("total_negative_reviews").fill_null(0),
#     pl.col("total_reviews").fill_null(0)
# )
# steam_games_agg

In [48]:
# read categories json into a table
with open("../data/categories.json") as f:
    categories = json.load(f)
cat_df = pl.DataFrame(categories).select(["categoryid", "name"]).rename({"name": "category"})
cat_df

categoryid,category
i64,str
2,"""Single-player"""
1,"""Multi-player"""
20,"""MMO"""
50,"""High-Quality Audio"""
49,"""PvP"""
…,…
44,"""Remote Play Together"""
61,"""HDR"""
62,"""Family Sharing"""


In [33]:
steam_apps = pl.read_parquet("../data/raw/games/steam_games_full.parquet").lazy()
steam_apps.limit(5).collect()

appid,name,type,required_age,is_free,minimum_pc_requirements,recommended_pc_requirements,controller_support,detailed_description,about_the_game,short_description,supported_languages,header_image,developers,publishers,price,category_ids,genres_list,windows_support,mac_support,linux_support,release_date,coming_soon,recommendations,dlc,review_score,review_score_desc,total_positive_reviews,total_negative_reviews,total_reviews
i64,str,str,i64,bool,str,str,str,str,str,str,list[str],str,list[str],list[str],f64,list[i64],list[str],bool,bool,bool,str,bool,i64,list[i64],i64,str,i64,i64,i64
2852260,"""Pandarunium""","""game""",0,false,"""Minimum:Requires a 64-bit proc…","""Recommended:Requires a 64-bit …",null,"""<strong>Pandarunium</strong> i…","""<strong>Pandarunium</strong> i…","""Team up with friends in this t…","[""English""]","""https://shared.akamai.steamsta…","[""Tenax Studios""]","[""Tenax Studios""]",3.99,"[2, 1, … 62]","[""Casual"", ""Indie""]",true,false,false,"""20 Jun, 2024""",false,null,[],0,"""2 user reviews""",2,0,2
1226222,"""Killer Gin Cats and Dogs DLC""","""dlc""",0,true,"""Minimum:OS *: Windows 7/8/8.1/…",null,"""full""","""This DLC package allows you to…","""This DLC package allows you to…","""This DLC package allows you to…","[""English""]","""https://shared.akamai.steamsta…","[""The Killer Gin""]",[],null,"[2, 21, … 62]","[""RPG""]",true,true,true,"""12 Mar, 2020""",false,null,[],0,"""No reviews""",0,0,0
291550,"""Brawlhalla""","""game""",0,true,"""Minimum:Memory: 2 GB RAMStorag…","""Recommended:Memory: 4 GB RAMNe…",null,"""<img class=""bb_img"" src=""https…","""<img class=""bb_img"" src=""https…","""An epic platform fighter for u…","[""English"", ""French"", … ""Turkish""]","""https://shared.akamai.steamsta…","[""Blue Mammoth Games""]","[""Ubisoft""]",null,"[2, 1, … 44]","[""Action"", ""Indie"", ""Free To Play""]",true,true,false,"""17 Oct, 2017""",false,2890,"[298641, 2974360, … 3392630]",8,"""Very Positive""",1306,266,1572
2116250,"""Draft Day Sports: College Foot…","""game""",0,false,"""Minimum:Requires a 64-bit proc…","""Recommended:Requires a 64-bit …",null,"""Prepare to lead your favorite …","""Prepare to lead your favorite …","""Draft Day Sports: College Foot…","[""English""]","""https://shared.akamai.steamsta…","[""Wolverine Studios""]","[""Wolverine Studios""]",53.0,"[2, 62]","[""Indie"", ""Simulation"", … ""Strategy""]",true,false,false,"""31 Aug, 2022""",false,null,[],5,"""Mixed""",9,6,15
946610,"""Pocket Rogues""","""game""",0,false,"""Minimum:Requires a 64-bit proc…","""Recommended:Requires a 64-bit …","""full""","""<p class=""bb_paragraph""><stron…","""<p class=""bb_paragraph""><stron…","""Pocket Rogues is a dynamic 2D-…","[""English"", ""Russian"", … ""Portuguese - Brazil""]","""https://shared.akamai.steamsta…","[""EtherGaming""]","[""EtherGaming""]",23.0,"[2, 1, … 62]","[""Action"", ""Adventure"", … ""Early Access""]",true,false,false,"""25 Oct, 2018""",false,453,[],8,"""Very Positive""",95,21,116


In [34]:
steam_apps.group_by(col("type")).len().collect()

type,len
str,u32
"""music""",544
"""dlc""",33735
"""video""",473
"""advertising""",138
"""mod""",89
"""game""",101291
"""series""",50
"""demo""",5793


In [35]:
# Reduce the scope to only games
steam_games = steam_apps.filter(col("type") == "game")

In [36]:
# Found duplicates
steam_games.group_by("name").len().filter(col("len") > 1).sort(by="len", descending=True).collect()

name,len
str,u32
"""Shadow of the Tomb Raider: Def…",20
"""No Way Out""",6
"""Aurora""",6
"""Escape""",5
"""Tom Clancy's Rainbow Six® Sieg…",5
…,…
"""Dystopia""",2
"""The Contact""",2
"""Lines""",2


In [56]:
games_table = steam_games.sort(["name", "total_reviews"], descending=True).unique(subset=["name"], keep="first").collect()

In [58]:
games_with_cat = games_table.explode("category_ids").rename({"category_ids": "categoryid"}).join(cat_df, on="categoryid", how="left") \
    .group_by("appid").agg(col("category").explode().alias("categories"))
games_table = games_with_cat.join(games_table, on="appid", how="inner").drop("category_ids")

In [60]:
games_table.write_parquet("../data/preprocessed/games.parquet")

In [59]:
games_table

appid,categories,name,type,required_age,is_free,minimum_pc_requirements,recommended_pc_requirements,controller_support,detailed_description,about_the_game,short_description,supported_languages,header_image,developers,publishers,price,genres_list,windows_support,mac_support,linux_support,release_date,coming_soon,recommendations,dlc,review_score,review_score_desc,total_positive_reviews,total_negative_reviews,total_reviews
i64,list[str],str,str,i64,bool,str,str,str,str,str,str,list[str],str,list[str],list[str],f64,list[str],bool,bool,bool,str,bool,i64,list[i64],i64,str,i64,i64,i64
2639280,"[""Single-player"", ""Family Sharing""]","""𣸩""","""game""",0,false,"""Minimum:Storage: 1 GB availabl…",null,null,"""<img class=""bb_img"" src=""https…","""<img class=""bb_img"" src=""https…","""《袖珍小说游戏》系列的首部作品，讲述失去了想象能力的作家，和…","[""Simplified Chinese"", ""Traditional Chinese""]","""https://shared.akamai.steamsta…","[""箱崎奈绪(Hakozaki Nao)""]","["" 箱崎奈绪(Hakozaki Nao)""]",5.03,"[""Adventure"", ""Casual"", ""Indie""]",true,false,false,"""29 Dec, 2023""",false,null,[],0,"""No reviews""",0,0,0
576960,"[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","""🧠 OUT OF THE BOX""","""game""",17,false,"""Minimum:Requires a 64-bit proc…","""Recommended:Requires a 64-bit …",null,"""<img class=""bb_img"" src=""https…","""<img class=""bb_img"" src=""https…","""Dive into a wild story of gang…","[""English"", ""Spanish - Spain"", … ""German""]","""https://shared.akamai.steamsta…","[""Nuclear Tales""]","[""🚀 Raiser Games""]",39.5,"[""Adventure"", ""Casual"", … ""Strategy""]",true,true,true,"""19 Jul, 2018""",false,202,[],6,"""Mostly Positive""",41,11,52
965340,"[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","""🚀 Human Rocket Person""","""game""",0,false,"""Minimum:OS *: Windows 7, Windo…",null,"""full""","""<img class=""bb_img"" src=""https…","""<img class=""bb_img"" src=""https…","""Human Rocket Person is an absu…","[""English"", ""German"", … ""Simplified Chinese""]","""https://shared.akamai.steamsta…","[""2nd Studio""]","[""2nd Studio""]",5.0,"[""Action"", ""Indie"", ""Simulation""]",true,false,false,"""14 Nov, 2018""",false,null,[],7,"""Positive""",12,1,13
460250,"[""Single-player"", ""Steam Achievements"", ""Steam Cloud""]","""🔴 Circles""","""game""",0,true,"""Minimum:OS *: Windows XP, Vist…",null,null,"""Circles is a unique, intuitive…","""Circles is a unique, intuitive…","""Circles is an abstract puzzle …","[""English"", ""French"", … ""Swedish""]","""https://shared.akamai.steamsta…","[""Jeroen Wimmers""]","[""Jeroen Wimmers""]",null,"[""Casual"", ""Indie""]",true,true,false,"""17 Feb, 2017""",false,null,[],7,"""Positive""",24,2,26
2919560,"[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","""👑Idle Calibur：Zero💕（选王之剑：零）""","""game""",0,false,"""Minimum:Requires a 64-bit proc…","""Recommended:Requires a 64-bit …",null,"""<h1>If you encounter performan…","""<p class=""bb_paragraph""><img c…","""A game that seamlessly integra…","[""Simplified Chinese"", ""English"", ""Japanese""]","""https://shared.akamai.steamsta…","[""绝汪""]","[""绝汪"", ""NPC Entertainment""]",24.0,"[""Indie"", ""RPG"", … ""Strategy""]",true,false,false,"""31 Oct, 2024""",false,405,[3114710],0,"""9 user reviews""",5,4,9
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3491960,"[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]",""" Lost Liminal""","""game""",0,false,"""Minimum:Requires a 64-bit proc…","""Recommended:Requires a 64-bit …",null,"""<p class=""bb_paragraph"">Explor…","""<p class=""bb_paragraph"">Explor…","""Lost Liminal is a quirky first…","[""English"", ""Japanese"", ""Simplified Chinese""]","""https://shared.akamai.steamsta…","[""Tuji Game""]","[""Tuji Game""]",null,"[""Action"", ""Adventure"", … ""RPG""]",true,false,false,"""Coming soon""",true,null,[],0,"""No reviews""",0,0,0
3440120,"[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]",""" Knights of the Crusades""","""game""",0,false,""

In [61]:
del games_table

In [3]:
steam_reviews = pl.read_parquet("../data/raw/reviews/steam_reviews_*.parquet")
steam_reviews

rec_id,author_id,appid,playtime_forever,playtime_last_two_weeks,playtime_at_review,num_games_owned,num_reviews,last_played,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,bool,i64,i64,f64,i64,bool,bool,bool,bool
190502887,76561198798149868,2852260,5,null,5,78,1,1742257396,"""english""","""I like the part with the panda""",1742257432,1742257432,true,0,0,0.5,0,true,false,false,false
168888656,76561198140769382,2852260,121,null,121,null,1,1719510337,"""english""","""Solo play works, but friendly …",1720019968,1720019968,true,3,0,0.565217,0,true,false,false,false
196678928,76561199086310241,291550,2202,null,2202,1,2,1607922050,"""english""","""Worst teammate matchmaking I'v…",1749353719,1749353719,true,0,0,0.5,0,true,false,false,false
192763979,76561198366424828,291550,75846,4123,62093,null,1,1749375912,"""english""","""good way to spend time if you …",1744671161,1744671161,true,0,0,0.5,0,true,false,false,false
189643572,76561199731966768,291550,699,null,232,null,1,1745115147,"""english""","""This is an overall good game. …",1741398352,1741398352,true,0,0,0.5,0,true,false,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
6418339,76561198065871127,222880,1554,null,418,227,8,1535835751,"""english""","""good""",1367079935,1367079935,true,0,0,0.5,0,true,false,false,false
6544900,76561198068660723,222880,569,null,58,null,7,1555809438,"""english""","""If you like counter strike you…",1367040549,1367040549,true,0,0,0.5,0,true,false,false,false
2732737,76561198011795478,222880,5574,null,778,null,45,1652131712,"""english""","""It is a good game. i dont know…",1367024720,1367024720,true,1,0,0.5029,0,true,false,false,false


In [4]:
deduplicated = steam_reviews.unique(subset=["author_id", "appid"], maintain_order=False)

In [5]:
deduplicated

rec_id,author_id,appid,playtime_forever,playtime_last_two_weeks,playtime_at_review,num_games_owned,num_reviews,last_played,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,bool,i64,i64,f64,i64,bool,bool,bool,bool
163524147,76561198407757564,1966720,6726,null,2245,null,4,1734228838,"""english""","""Cant wait to see what they wil…",1713993377,1713993377,true,0,0,0.5,0,true,false,true,false
177397292,76561198027049751,812140,626,null,626,null,133,1729422957,"""english""","""So, sadly I can not recommend …",1729423540,1729423540,false,1,0,0.5,0,true,false,false,false
146378199,76561199130298382,1959390,139,null,139,null,39,1694841073,"""english""","""i liked it quite a bit! I've h…",1694841481,1694841481,true,0,0,0.5,1,true,false,false,false
55077859,76561198074841893,914620,7295,null,1224,null,7,1644677449,"""english""","""Its a good Game but you alway…",1568303571,1568303571,false,1,1,0.392157,0,true,false,true,false
137580431,76561199048332868,585420,4922,38,201,75,17,1749279661,"""english""","""yes""",1682891619,1682891619,true,0,0,0.5,0,true,false,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
124406689,76561198088231107,1930600,701,null,525,null,89,1682473019,"""english""","""Waifus > Vampires""",1666771524,1666771524,true,0,0,0.5,0,true,false,true,false
101916016,76561197964936336,1466860,17970,null,1495,null,6,1692517653,"""english""","""Awesome game! Made some big no…",1635629157,1635629157,true,0,0,0.5,0,true,false,false,false
120471323,76561198381440680,22380,6176,null,1507,null,36,1716500989,"""english""","""I'm studying better because of…",1660265519,1660265519,true,1,0,0.52381,0,true,false,false,false


In [6]:
user_counts = deduplicated.group_by("author_id").agg(pl.len().alias("num_reviews")).filter(col("num_reviews") >= 10).select("author_id")
game_counts = deduplicated.group_by("appid").agg(pl.len().alias("num_reviews")).filter(col("num_reviews") >= 10).select("appid")

In [7]:
reviews_table = deduplicated.join(user_counts, on="author_id", how="inner").join(game_counts, on="appid", how="inner")

In [8]:
reviews_table.drop(["num_games_owned", "num_reviews"]).write_parquet("../data/preprocessed/reviews.parquet")

In [22]:
# users table. About half of num_games_owned is null, num_reviews has no nulls
users_table = reviews_table.select(["author_id", "num_games_owned", "num_reviews"]).sort(["author_id", "num_games_owned", "num_reviews"], descending=True).unique(subset=["author_id"], keep="first")

In [23]:
users_table.write_parquet("../data/preprocessed/users.parquet")

In [30]:
del reviews_table
del deduplicated
del users_table

NameError: name 'reviews_table' is not defined